In [ ]:
# Import the predictions and seperate between training1, training2, and testing

import pandas as pd
import numpy as np

import os

DATA_DIR_NAME = "researchData"
DATA_COL_NAME = "Electricity:Facility [kW](Hourly)"

from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import math

import tensorflow as tf
from tensorflow import keras
import keras.backend as K

import warnings
import time
 
DATA_DIR_NAME = "input"
DATA_COL_NAME = "Electricity:Facility [kW](Hourly)"

TESLA_DATA_DIR_NAME = "teslaData"

warnings.filterwarnings("ignore")

# LSTM Code below derived from https://machinelearningmastery.com/
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df
 
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)
 
# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]
 
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled
 
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]
 
# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, shuffle=False)
        model.reset_states()
    return model
 
# make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]


#LENARD MODEL
dir_path = os.getcwd()
directory = os.path.join(dir_path, DATA_DIR_NAME)
for root,dirs,files in os.walk(directory):
    for file in files:
        
        print(file)

        df = pd.read_csv(DATA_DIR_NAME + '/' + file)
        
        full = df[DATA_COL_NAME]
        split_point = int(len(full) * .8)
        
        training_learners = full[:split_point]
        testing_learners = full[split_point:]
        
        training_lenard = testing_learners[:int(len(testing_learners) * .8)]
        testing_lenard  = testing_learners[int(len(testing_learners) * .8):]

        
        # ARIMA
        arima_model = pm.auto_arima(training_learners, start_p=1, start_q=1,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=3, max_q=3, # maximum p and q
                          m=24,              # frequency of series
                          d=0,           # let model determine 'd'
                          seasonal=False,   # Seasonality
                          start_P=0, 
                          D=0, 
                          trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True,
                          n_jobs=-1)
        
        arima_model = ARIMA(training_learners, order=arima_model.order)
        arima_model_fit = arima_model.fit(disp=0)   

        arima_yhat = arima_model_fit.predict(start=0, end=len(testing_learners) - 1)
        
        
        # Holt Winters
        trend = ["add", "mul"]
        damped = [True, False]
        seasonal = ["add", "mul"]
        seasonal_periods = [24, 168]
        
        best_mse = 100
        
        for trend_x in trend:
            for damped_x in damped:
                for seasonal_x in seasonal:
                    for seasonal_periods_x in seasonal_periods:
                        hw_model = ExponentialSmoothing(training_learners,
                                                        trend=trend_x,
                                                        seasonal=seasonal_x,
                                                        seasonal_periods=seasonal_periods_x, 
                                                        damped=damped_x)   
                        
                        hw_model_fit = hw_model.fit()
                        
                        new_hw_yhat = hw_model_fit.predict(start=0, end=len(testing_learners) - 1)
                        
                        hw_mse = mean_squared_error(new_hw_yhat, testing_learners)
                        
                        if(hw_mse < best_mse):
                            hw_yhat = new_hw_yhat
                            best_mse = hw_mse    
    
        
        # LSTM
        lstm_split_point = int(len(full) * .2)
        
        diff_values = difference(full, 1)

        supervised = timeseries_to_supervised(diff_values, 1)
        supervised_values = supervised.values
    
        lstm_train, lstm_test = supervised_values[0:-lstm_split_point:], supervised_values[-lstm_split_point:]
 
        # transform the scale of the data
        scaler, train_scaled, test_scaled = scale(lstm_train, lstm_test)
 
        # fit the model
        lstm_model = fit_lstm(train_scaled, 1, 10, 2)
        # forecast the entire training dataset to build up state for forecasting
        train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
        lstm_model.predict(train_reshaped, batch_size=1)
 
        # walk-forward validation on the test data
        lstm_yhat = list()
        for i in range(len(test_scaled)):
            # make one-step forecast
            X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
            yhat = forecast_lstm(lstm_model, 1, X)
            # invert scaling
            yhat = invert_scale(scaler, X, yhat)
            # invert differencing
            yhat = inverse_difference(df[DATA_COL_NAME].values, yhat, len(test_scaled)+1-i)
            # store forecast
            lstm_yhat.append(yhat)
            
            
        # Import Tesla predictions from matlab
#         tesla_df = pd.read_csv(TESLA_DATA_DIR_NAME + '/' + file)
#         tesla_yhat = tesla_df.index

        # Persistence
        persistence_yhat = full[split_point - 24:len(full) - 24]
        
        split_point = int(len(arima_yhat) * .8)
    
        train_house = np.array([arima_yhat[:split_point], # Training data
                                hw_yhat[:split_point],
                                lstm_yhat[:split_point],
                                persistence_yhat[:split_point]])
    
        train_label = np.array(training_lenard)  # Training validation data
    
        test_house = np.array([arima_yhat[split_point:],  # Testing data
                               hw_yhat[split_point:],
                               lstm_yhat[split_point:],
                               persistence_yhat[split_point:]])
        
        test_label = np.array(testing_lenard)
        
        x = np.array([[1]])
        
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(400, input_shape=(1,), activation='relu'),
            tf.keras.layers.Dense(200, activation='relu'),
            tf.keras.layers.Dense(100, activation='relu'),
            tf.keras.layers.Dense(4, activation='sigmoid')
        ])
        
            # The wrapper is used to import the real training and validation data to the custom loss function
        def custom_loss_wrapper(curr_house, curr_label) :
    
            # Uses the imported data to apply the outputted weights and use the mse between the real values and 
            # the summed prediction as the loss
            def custom_loss(y_actual, y_predicted) :
            
                new_arima = curr_house[0] * y_predicted[0][0]
                new_holt = curr_house[1] * y_predicted[0][1]
                new_lstm = curr_house[2] * y_predicted[0][2]
                new_persistence = curr_house[3] * y_predicted[0][3]
                #new_tesla = curr_house[4] * y_predicted[0][4]
    
                new = tf.math.add(new_arima, new_holt)
                new = tf.math.add(new, new_lstm)
                new = tf.math.add(new, new_persistence)
                #new = tf.math.add(new, new_tesla)
            
                return K.mean(K.square(new - curr_label))

            return custom_loss
    
    
        model.compile(optimizer='adam', 
                      loss=custom_loss_wrapper(tf.Variable(train_house, dtype=tf.float32), 
                                               tf.Variable(train_label, dtype=tf.float32)))
    
        # Trains the model using the first 80% of the learner's predictions
        model.fit(x, x, epochs=500, verbose=0)
    
        model.compile(optimizer='adam',
                      loss=custom_loss_wrapper(tf.Variable(test_house, dtype=tf.float32), 
                                               tf.Variable(test_label, dtype=tf.float32)))
    
        # Calculates the mse between the actual values and the prediction found using the calculated weights
        test_loss = model.evaluate(x, x, verbose=2)

        print('\nTest loss:', test_loss)
    
#         mse_list.append(test_loss)
#         weights.append(model.predict(x)[0])
        print(model.predict(x)[0])